In [ ]:
from google.colab import files
import pandas as pd

# آپلود فایل‌ها
uploaded = files.upload()

# دیکشنری برای نگهداری دیتافریم‌ها
dfs = {}

for filename in uploaded.keys():
    # لود کردن فایل‌های Excel
    df = pd.read_excel(filename)
    dfs[filename] = df
    print(f"✅ {filename} لود شد")
    print("اولین ردیف‌ها:")
    print(df.head())
    print("اطلاعات کلی:")
    print(df.info())
    print("آمار توصیفی:")
    print(df.describe())
    print("-"*50)


Saving cornweek2021.xlsx to cornweek2021.xlsx
Saving dxyweek2021.xlsx to dxyweek2021.xlsx
Saving ngweek2021.xlsx to ngweek2021.xlsx
Saving oil week2021.xlsx to oil week2021.xlsx
Saving uraaweek2021.xlsx to uraaweek2021.xlsx
Saving wheat week2021.xlsx to wheat week2021.xlsx
✅ cornweek2021.xlsx لود شد
اولین ردیف‌ها:
        Date   Price   Open    High     Low     Vol.  Change %
0 2025-08-31  418.00  419.5  424.90  414.10  729.65K   -0.0054
1 2025-08-24  420.25  413.0  420.50  403.50    1.15M    0.0213
2 2025-08-17  411.50  404.5  413.75  401.25  890.30K    0.0154
3 2025-08-10  405.25  405.0  409.25  392.00    1.28M   -0.0006
4 2025-08-03  405.50  411.0  412.75  396.75  988.12K   -0.0128
اطلاعات کلی:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      244 non-null    datetime64[ns]
 1   Price     244 non-null    float64       
 2   

In [ ]:
import pandas as pd
import numpy as np
from functools import reduce

# لیست فایل‌ها و نام مناسب برای rename ستون‌ها
files = {
    "cornweek2021.xlsx": "corn",
    "dxyweek2021.xlsx": "dxy",
    "ngweek2021.xlsx": "ng",
    "oil week2021.xlsx": "oil",
    "uraaweek2021.xlsx": "urea",
    "wheat week2021.xlsx": "wheat"
}

dfs = {}
for f, name in files.items():
    df = pd.read_excel(f, parse_dates=['Date'])
    # پاکسازی Vol.
    if 'Vol.' in df.columns:
        df['Vol.'] = df['Vol.'].astype(str).str.replace('K','e3').str.replace('M','e6')
        df['Vol.'] = pd.to_numeric(df['Vol.'], errors='coerce')
    # تغییر نام ستون‌ها به جز Date
    df = df.rename(columns={col: f"{name}_{col}" for col in df.columns if col != 'Date'})
    dfs[name] = df

# merge همه دیتافریم‌ها روی ستون Date
merged_df = reduce(lambda left, right: pd.merge(left, right, on='Date', how='outer'), dfs.values())

# مرتب سازی بر اساس Date
merged_df = merged_df.sort_values('Date').reset_index(drop=True)

# پر کردن مقادیر گمشده
merged_df = merged_df.fillna(method='ffill')

# نمایش
print(merged_df.head())
print(merged_df.info())


        Date  corn_Price  corn_Open  corn_High  corn_Low  corn_Vol.  \
0 2021-01-03      496.25     487.25     502.75    479.50  1040000.0   
1 2021-01-10      531.50     496.50     541.50    489.25  1130000.0   
2 2021-01-17      500.50     531.50     534.25    499.25   865460.0   
3 2021-01-24      547.00     500.00     553.75    492.50  1250000.0   
4 2021-01-31      548.50     549.00     558.00    536.50  1040000.0   

   corn_Change %  dxy_Price  dxy_Open  dxy_High  ...  urea_High  urea_Low  \
0         0.0253        NaN       NaN       NaN  ...      288.0     288.0   
1         0.0710        NaN       NaN       NaN  ...      293.0     293.0   
2        -0.0583        NaN       NaN       NaN  ...      302.0     302.0   
3         0.0929        NaN       NaN       NaN  ...      371.5     371.5   
4         0.0027        NaN       NaN       NaN  ...      367.5     367.5   

   urea_Vol.  urea_Change %  wheat_Price  wheat_Open  wheat_High  wheat_Low  \
0        NaN         0.0647    

/tmp/ipython-input-555356804.py:33: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df = merged_df.fillna(method='ffill')


In [ ]:
merged_df = merged_df.ffill()

In [ ]:
import pandas as pd

# کپی برای کار روی دیتافریم
df = merged_df.copy()

# ساخت ویژگی‌های زمانی
df['Week'] = df['Date'].dt.isocalendar().week
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

# ساخت ویژگی‌های گذشته نگر Corn (1 تا 3 هفته قبل)
for lag in range(1, 4):
    df[f'corn_Price_lag_{lag}'] = df['corn_Price'].shift(lag)

# ساخت ویژگی‌های تغییرات درصد هفته قبل
df['corn_Change_lag_1'] = df['corn_Change %'].shift(1)

# پر کردن NaN بعد از shift
df = df.ffill()

# نمایش نمونه داده‌ها
print(df.head())


        Date  corn_Price  corn_Open  corn_High  corn_Low  corn_Vol.  \
0 2021-01-03      496.25     487.25     502.75    479.50  1040000.0   
1 2021-01-10      531.50     496.50     541.50    489.25  1130000.0   
2 2021-01-17      500.50     531.50     534.25    499.25   865460.0   
3 2021-01-24      547.00     500.00     553.75    492.50  1250000.0   
4 2021-01-31      548.50     549.00     558.00    536.50  1040000.0   

   corn_Change %  dxy_Price  dxy_Open  dxy_High  ...  wheat_Low  wheat_Vol.  \
0         0.0253        NaN       NaN       NaN  ...     634.12         NaN   
1         0.0710        NaN       NaN       NaN  ...     628.88         NaN   
2        -0.0583        NaN       NaN       NaN  ...     633.12         NaN   
3         0.0929        NaN       NaN       NaN  ...     625.75         NaN   
4         0.0027        NaN       NaN       NaN  ...     625.75         NaN   

   wheat_Change %  Week  Month  Year  corn_Price_lag_1  corn_Price_lag_2  \
0         -0.0051    5

In [ ]:
# لیست ستون‌هایی که میخوایم lag بسازیم
features = ['dxy_Price', 'dxy_Change %',
            'oil_Price', 'oil_Change %',
            'ng_Price', 'ng_Change %',
            'urea_Price', 'urea_Change %',
            'wheat_Price', 'wheat_Change %']

# ساخت lag 1 تا 3 هفته برای هر ستون
for col in features:
    for lag in range(1, 4):
        df[f'{col}_lag_{lag}'] = df[col].shift(lag)

# پر کردن NaN های ایجاد شده
df = df.ffill()

# نمایش نمونه داده‌ها
print(df.head())


        Date  corn_Price  corn_Open  corn_High  corn_Low  corn_Vol.  \
0 2021-01-03      496.25     487.25     502.75    479.50  1040000.0   
1 2021-01-10      531.50     496.50     541.50    489.25  1130000.0   
2 2021-01-17      500.50     531.50     534.25    499.25   865460.0   
3 2021-01-24      547.00     500.00     553.75    492.50  1250000.0   
4 2021-01-31      548.50     549.00     558.00    536.50  1040000.0   

   corn_Change %  dxy_Price  dxy_Open  dxy_High  ...  urea_Price_lag_3  \
0         0.0253        NaN       NaN       NaN  ...               NaN   
1         0.0710        NaN       NaN       NaN  ...               NaN   
2        -0.0583        NaN       NaN       NaN  ...               NaN   
3         0.0929        NaN       NaN       NaN  ...             288.0   
4         0.0027        NaN       NaN       NaN  ...             293.0   

   urea_Change %_lag_1  urea_Change %_lag_2  urea_Change %_lag_3  \
0                  NaN                  NaN                 

In [ ]:
# ========================
# 1️⃣ تعریف تارگت (Target)
# ========================
# هدف: پیش‌بینی قیمت اوره هفته بعد
df['urea_Price_next'] = df['urea_Price'].shift(-1)  # قیمت هفته بعد

In [ ]:
# ========================
# 2️⃣ حذف ردیف‌های ناقص
# ========================
df = df.dropna()  # همه ردیف‌های NaN حذف می‌شوند

In [ ]:
# ========================
# 3️⃣ انتخاب Features و Target
# ========================
# فرض کنیم تمام ستون‌های lag اوره، corn، wheat و تغییرات‌شون رو استفاده کنیم
feature_cols = [col for col in df.columns if 'lag' in col or 'Change' in col]
target_col = 'urea_Price_next'

X = df[feature_cols]
y = df[target_col]

In [ ]:
from sklearn.model_selection import train_test_split
# ========================
# 4️⃣ تقسیم دیتا به Train/Test
# ========================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)  # چون سری زمانی است، shuffle=False مهمه

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)


Shape of X_train: (27, 40)
Shape of X_test: (7, 40)
Shape of y_train: (27,)
Shape of y_test: (7,)


In [ ]:
import pandas as pd
from google.colab import files

# آپلود فایل‌ها
uploaded = files.upload()

# لود داده‌ها
dfs_new = {}
for filename in uploaded.keys():
    df = pd.read_excel(filename, parse_dates=True)
    dfs_new[filename] = df
    print(f"✅ {filename} لود شد")
    print(df.head(), "\n")
    print(df.info(), "\n")
    print(df.describe(), "\n")


Saving urea daily 2017.xlsx to urea daily 2017.xlsx
Saving urea weekly 2017 (1).xlsx to urea weekly 2017 (1).xlsx
✅ urea daily 2017.xlsx لود شد
        Date  Price   Open   High    Low Vol.  Change %
0 2025-09-15  427.5  427.5  427.5  427.5  NaN    0.0000
1 2025-09-12  427.5  427.5  427.5  427.5  NaN    0.0000
2 2025-09-11  427.5  427.5  427.5  427.5  NaN   -0.0023
3 2025-09-10  428.5  428.5  428.5  428.5  NaN   -0.0058
4 2025-09-09  431.0  431.0  431.0  431.0  NaN    0.0117 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1986 entries, 0 to 1985
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      1986 non-null   datetime64[ns]
 1   Price     1986 non-null   float64       
 2   Open      1986 non-null   float64       
 3   High      1986 non-null   float64       
 4   Low       1986 non-null   float64       
 5   Vol.      106 non-null    object        
 6   Change %  1986 non-null   float64      